In [205]:
import pandas as pd
df=pd.read_csv(r"C:\Users\Lenovo\Desktop\projet_baina\nouveau_projet\file (1).csv")

In [206]:
df.shape

(798, 20)

In [207]:
df1=df.copy()

In [208]:
df1=df1.drop_duplicates()
df1=df1.reset_index(drop=True)
df1.shape

(798, 20)

In [209]:
df1=df1.drop(["Unnamed: 0","country_from","country_to"],axis=1)

dropithom hyt db matalan country from rah kata3tyna nefs ma3loma d dawru , 2la 3rfna rah msha preimier league donc rah msha l england . hyt kol dawla fyha ghy dwru wahd 

In [210]:
occurrences_clubs = df1["club_from"].value_counts().unique()
occurrences_clubs

array([15, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1], dtype=int64)

In [211]:
df1["club_from"].unique()

array(['Borussia Dortmund', 'Ajax Amsterdam', 'Leicester City',
       'AS Monaco', 'Real Madrid', 'Real Sociedad', 'Juventus FC',
       'Manchester City', 'Liverpool FC', 'Chelsea FC',
       'Brighton & Hove Albion', 'ACF Fiorentina', 'Sevilla FC',
       'Leeds United', 'Everton FC', 'SL Benfica', 'SSC Napoli',
       'Atalanta BC', 'Valencia CF', 'Manchester United', 'Bayern Munich',
       'Sporting CP', 'AC Milan', 'CA River Plate', 'Olympique Lyon',
       'OGC Nice', 'FC Porto', 'LOSC Lille', 'Club Brugge KV',
       'Torino FC', 'Stade Rennais FC', 'Red Bull Salzburg',
       'SC Freiburg', 'Tottenham Hotspur', 'US Sassuolo', 'Fulham FC',
       'Atlético de Madrid', 'RB Leipzig', 'Wolverhampton Wanderers',
       'Stade Reims', 'SS Lazio', 'VfL Wolfsburg', 'TSG 1899 Hoffenheim',
       'Feyenoord Rotterdam', 'VfB Stuttgart', 'Arsenal FC',
       'RCD Espanyol Barcelona', 'Olympique Marseille', 'Burnley FC',
       'Eintracht Frankfurt', 'Rangers FC', 'RC Lens',
       'Manch

In [212]:
colonnes_categorielles1 = ['league_from', 'league_to']

for colonne in colonnes_categorielles1:
    counts = df1[colonne].value_counts()
    categories_autre = counts[counts <4].index.tolist()
    df1[colonne] = df1[colonne].apply(lambda x: 'Autre' if x in categories_autre else x)
colonnes_categorielles2= ['club_to', 'club_from']
for colonne in colonnes_categorielles2:
    counts = df1[colonne].value_counts()
    categories_autre = counts[counts < 11].index.tolist()
    if 'Saudi Pro League' in categories_autre:
        categories_autre.remove('Saudi Pro League')
    
    df1[colonne] = df1[colonne].apply(lambda x: 'Autre' if x in categories_autre else x)

df_encoded = pd.get_dummies(df1, columns=colonnes_categorielles1 + colonnes_categorielles2 + ['position'] + ['loan'], dtype=int)

In [213]:
df_encoded.shape

(798, 93)

In [214]:
df_encoded["fee"] = df_encoded["fee"].str.replace('k', '')
df_encoded["fee"]=df_encoded["fee"].astype(float)
df_encoded["market_value"] = df_encoded["market_value"].str.replace('k', '')
df_encoded["market_value"]=df_encoded["market_value"].astype(float)

In [215]:
df_encoded=df_encoded.dropna()
df_encoded=df_encoded.drop_duplicates()

In [216]:
df_encoded.iloc[:,4:11].corr().abs()

,ATT,SKI,MOV,POW,MEN,DEF,GK
ATT,1.000000,0.906219,0.653273,0.827636,0.840280,0.157337,0.783897
SKI,0.906219,1.000000,0.699717,0.714497,0.872538,0.285973,0.753965
MOV,0.653273,0.699717,1.000000,0.507865,0.559894,0.076811,0.563624
POW,0.827636,0.714497,0.507865,1.000000,0.781490,0.206922,0.603301
MEN,0.840280,0.872538,0.559894,0.781490,1.000000,0.528228,0.727101
DEF,0.157337,0.285973,0.076811,0.206922,0.528228,1.000000,0.444797
GK,0.783897,0.753965,0.563624,0.603301,0.727101,0.444797,1.000000


In [217]:
df_encoded=df_encoded.reset_index(drop=True)

In [218]:
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error , mean_absolute_error
from sklearn.model_selection import train_test_split
reg=LinearRegression()
X=df_encoded.drop(["fee","name"],axis=1)
y=df_encoded["fee"]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1937)
reg.fit(X_train,y_train)
y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)


Mean Squared Error: 5298.8040195393505
Mean Absolute Error: 46.52315498475019


In [219]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
X = df_encoded.drop(["fee", "name"], axis=1)
pca = PCA(n_components=10)  
X_pca = pca.fit_transform(X)
y = df_encoded["fee"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1937)
alphas = [0.001, 0.01, 0.1, 1, 10, 100]
reg = Ridge()
param_grid = {'alpha': alphas}
grid_search = GridSearchCV(reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)
best_reg = grid_search.best_estimator_
y_pred = best_reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("Best alpha found:", best_reg.alpha)

Mean Squared Error: 4321.5589591766975
Mean Absolute Error: 37.11408750795968
Best alpha found: 10


In [220]:
df_encoded.iloc[:,4:11].corr()

,ATT,SKI,MOV,POW,MEN,DEF,GK
ATT,1.000000,0.906219,0.653273,0.827636,0.840280,0.157337,-0.783897
SKI,0.906219,1.000000,0.699717,0.714497,0.872538,0.285973,-0.753965
MOV,0.653273,0.699717,1.000000,0.507865,0.559894,0.076811,-0.563624
POW,0.827636,0.714497,0.507865,1.000000,0.781490,0.206922,-0.603301
MEN,0.840280,0.872538,0.559894,0.781490,1.000000,0.528228,-0.727101
DEF,0.157337,0.285973,0.076811,0.206922,0.528228,1.000000,-0.444797
GK,-0.783897,-0.753965,-0.563624,-0.603301,-0.727101,-0.444797,1.000000


In [221]:
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X = df_encoded.drop(["fee", "name"], axis=1)
y = df_encoded["fee"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1937)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lasso = LassoCV(cv=5, max_iter=10000)
lasso.fit(X_train_scaled, y_train)

y_pred = lasso.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("Best alpha found:", lasso.alpha_)


Mean Squared Error: 4248.738331819295
Mean Absolute Error: 28.833730459637938
Best alpha found: 5.492682386726116
